# INGESTA #

Este notebook tiene como propósito el aplanamiento de los datos suministrados en formato JSONL y el cargue de los datos remitidos .csv, para generar un ejercicio de carga en tablas de SQL Server simulando un proceso de ingesta de información a una base de datos, que posteriormente se va a consumir en un proceso de análisis de datos para la elaboración de un modelo de Machine Learning

*librerias*


Las librerías que se van a aplicar en el notebook

In [ ]:
import json
import pandas as pd
import pyodbc

Se utiliza la librería JSON para el aplanamiento del archivo JSONL, la opción de escalabilidad es PYSPARK, pero en este caso los datos remitidos se pueden mover sin problema con este paquete 

In [34]:
path = 'C:/Users/diegrica/OneDrive - colsubsidio.com/Documents/DARC/MELI/DATA/'

data_json = [json.loads(line) for line in open(path+'prints.json', 'r')]
df_prints = pd.json_normalize(data_json)
df_prints


,day,user_id,event_data.position,event_data.value_prop
0,2020-11-01,98702,0,cellphone_recharge
1,2020-11-01,98702,1,prepaid
2,2020-11-01,63252,0,prepaid
3,2020-11-01,24728,0,cellphone_recharge
4,2020-11-01,24728,1,link_cobro
...,...,...,...,...
508612,2020-11-30,50807,0,send_money
508613,2020-11-30,50807,1,prepaid
508614,2020-11-30,50807,2,credits_consumer
508615,2020-11-30,1487,0,point


Revisión de los datos cargados para la carga a SQL Server y compatibilidad con las tablas creadas

In [43]:
df_prints.dtypes

day                      object
user_id                   int64
event_data_position       int64
event_data_value_prop    object
dtype: object

Ajuste de los nombres de las columnas para que no existan novedades en el momento de la carga

In [35]:
df_prints.rename(columns={'event_data.position': 'event_data_position'}, inplace=True)
df_prints.rename(columns={'event_data.value_prop': 'event_data_value_prop'}, inplace=True)

Ajuste del tipo de dato de la columna

In [41]:
df_prints['user_id']= df_prints['user_id'].astype(object)
df_prints['event_data_position']= df_prints['event_data_position'].astype(int)

C:\Users\diegrica\AppData\Local\Temp\ipykernel_23748\1105427095.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prints['user_id']= prints['user_id'].astype(object)
C:\Users\diegrica\AppData\Local\Temp\ipykernel_23748\1105427095.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  prints['event_data_position']= prints['event_data_position'].astype(int)


Carga del segundo data frame de JSONL a Python

In [48]:
path = 'C:/Users/diegrica/OneDrive - colsubsidio.com/Documents/DARC/MELI/DATA/'

data_json = [json.loads(line) for line in open(path+'taps.json', 'r')]
df_taps = pd.json_normalize(data_json)
df_taps

,day,user_id,event_data.position,event_data.value_prop
0,2020-11-01,98702,0,cellphone_recharge
1,2020-11-01,3708,2,point
2,2020-11-01,3708,3,send_money
3,2020-11-01,93963,0,transport
4,2020-11-01,93963,1,cellphone_recharge
...,...,...,...,...
50854,2020-11-30,64149,2,credits_consumer
50855,2020-11-30,3329,1,point
50856,2020-11-30,77012,1,cellphone_recharge
50857,2020-11-30,14704,2,cellphone_recharge


Revisión de los datos cargados para la carga a SQL Server y compatibilidad con las tablas creadas

In [49]:
df_taps.dtypes

day                      object
user_id                   int64
event_data.position       int64
event_data.value_prop    object
dtype: object

Ajuste de los nombres de las columnas para que no existan novedades en el momento de la carga

In [50]:
df_taps.rename(columns={'event_data.position': 'event_data_position'}, inplace=True)
df_taps.rename(columns={'event_data.value_prop': 'event_data_value_prop'}, inplace=True)

Ajuste del tipo de dato de la columna

In [51]:
df_taps['user_id']= df_taps['user_id'].astype(object)
df_taps['event_data_position']= df_taps['event_data_position'].astype(int)

In [ ]:
df_pay = pd.read_csv('C:/Users/diegrica/OneDrive - colsubsidio.com/Documents/DARC/MELI/DATA/pays.csv')
df_pay

,pay_date,total,user_id,value_prop
0,2020-11-01,7.04,35994,link_cobro
1,2020-11-01,37.36,79066,cellphone_recharge
2,2020-11-01,15.84,19321,cellphone_recharge
3,2020-11-01,26.26,19321,send_money
4,2020-11-01,35.35,38438,send_money
...,...,...,...,...
756478,2020-11-30,17.76,74524,send_money
756479,2020-11-30,111.02,42594,transport
756480,2020-11-30,0.81,1338,transport
756481,2020-11-30,34.71,1338,credits_consumer


Revisión de los datos cargados para la carga a SQL Server y compatibilidad con las tablas creadas

In [ ]:
df_pay.dtypes

pay_date       object
total         float64
user_id        object
value_prop     object
dtype: object

Ajuste del tipo de dato de la columna

In [ ]:
df_pay['total'] = df_pay['total'].astype(float)
df_pay['user_id'] = df_pay['user_id'].astype(object)

Se realiza la conexión a SQL Server donde se van a ingestar las tablas para el posterior análisis de los datos

In [46]:
# Conexión a SQL Server
try: 
    conexion = pyodbc.connect (
    'DRIVER={ODBC Driver 17 for SQL Server};'
    'SERVER=BOGAL06DIEGRICA;'
    'DATABASE=Challenge;'
    'Trusted_Connection=yes;'
) 
    print('Conexion ok')

except:
    print('falla de conexion')

Conexion ok


In [ ]:
datos_para_sql = [tuple(x) for x in df_prints.values]

columnas = ', '.join([f'[{col}]' for col in df_prints.columns])
placeholders = ', '.join(['?' for _ in df_prints.columns])
sql_insert = f"INSERT INTO dbo.prints ({columnas}) VALUES ({placeholders})"

cursorInsert = conexion.cursor()
cursorInsert.executemany(sql_insert, datos_para_sql)

conexion.commit()

In [ ]:
datos_para_sql2 = [tuple(x) for x in df_taps.values]

columnas2 = ', '.join([f'[{col}]' for col in df_taps.columns])
placeholders2 = ', '.join(['?' for _ in df_taps.columns])
sql_insert2 = f"INSERT INTO dbo.prints ({columnas2}) VALUES ({placeholders2})"

cursorInsert = conexion.cursor()
cursorInsert.executemany(sql_insert2, datos_para_sql2)

conexion.commit()

In [52]:
datos_para_sql3 = [tuple(x) for x in df_pay.values]

columnas3 = ', '.join([f'[{col}]' for col in df_pay.columns])
placeholders3 = ', '.join(['?' for _ in df_pay.columns])
sql_insert3 = f"INSERT INTO dbo.pays ({columnas3}) VALUES ({placeholders3})"

cursorInsert = conexion.cursor()
cursorInsert.executemany(sql_insert3, datos_para_sql3)

conexion.commit()

Cierre de la conexión para evitar sesiones abiertas y consumo de recursos

In [ ]:
conexion.close()